### 目标  
完全用LLM来实现这篇阿联酋发的四大中用到的基于Joern的Program Slicing（程序切片）
https://www.usenix.org/system/files/usenixsecurity25-lekssays.pdf

程序切片：
https://www.cs.purdue.edu/homes/xyzhang/spring07/590F-slicing.pdf  
从目标变量（某一行代码）反向分析（backward analysis）找出所有和当前代码相关的，把这些代码提出来，得到一个只和目标变量相关的精简代码。


In [19]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
system_prompt = "You are a helpful program analysis expert."
MODEL = "qwen3-coder-plus"

def llm_chat(question, prompt=system_prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": question}
        ],
        stream=False
    )
    return response.choices[0].message.content

llm_chat("Who are you?")

'I am Qwen, a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. I am capable of answering questions, creating text such as stories, official documents, emails, and scripts, performing logical reasoning, coding, and more. I can also express opinions and play games. I support multiple languages, including Chinese, English, German, French, Spanish, Portuguese, Italian, Dutch, Russian, Czech, Polish, Arabic, Persian, Hebrew, Turkish, Japanese, Korean, and many others.'

In [18]:
# One Sample C Code for Testing Program Slicing
raw_c_code = """
static void dma_rx(struct b43_dmaring *ring, int *slot)
{
	const struct b43_dma_ops *ops = ring->ops;
	struct b43_dmadesc_generic *desc;
	struct b43_dmadesc_meta *meta;
	struct b43_rxhdr_fw4 *rxhdr;
	struct sk_buff *skb;
	u16 len;
	int err;
	dma_addr_t dmaaddr;
	desc = ops->idx2desc(ring, *slot, &meta);
	sync_descbuffer_for_cpu(ring, meta->dmaaddr, ring->rx_buffersize);
	skb = meta->skb;
	rxhdr = (struct b43_rxhdr_fw4 *)skb->data;
	len = le16_to_cpu(rxhdr->frame_len);
	if (len == 0) {
		int i = 0;
		do {
			udelay(2);
			barrier();
			len = le16_to_cpu(rxhdr->frame_len);
		} while (len == 0 && i++ < 5);
		if (unlikely(len == 0)) {
			dmaaddr = meta->dmaaddr;
			goto drop_recycle_buffer;
		}
	}
	if (unlikely(b43_rx_buffer_is_poisoned(ring, skb))) {
		/* Something went wrong with the DMA.
		 * The device did not touch the buffer and did not overwrite the poison. */
		b43dbg(ring->dev->wl, "DMA RX: Dropping poisoned buffer.\n");
		dmaaddr = meta->dmaaddr;
		goto drop_recycle_buffer;
	}
	if (unlikely(len + ring->frameoffset > ring->rx_buffersize)) {
		/* The data did not fit into one descriptor buffer
		 * and is split over multiple buffers.
		 * This should never happen, as we try to allocate buffers
		 * big enough. So simply ignore this packet.
		 */
		int cnt = 0;
		s32 tmp = len;
		while (1) {
			desc = ops->idx2desc(ring, *slot, &meta);
			/* recycle the descriptor buffer. */
			b43_poison_rx_buffer(ring, meta->skb);
			sync_descbuffer_for_device(ring, meta->dmaaddr,
						   ring->rx_buffersize);
			*slot = next_slot(ring, *slot);
			cnt++;
			tmp -= ring->rx_buffersize;
			if (tmp <= 0)
				break;
		}
		b43err(ring->dev->wl, "DMA RX buffer too small "
		       "(len: %u, buffer: %u, nr-dropped: %d)\n",
		       len, ring->rx_buffersize, cnt);
		goto drop;
	}
	dmaaddr = meta->dmaaddr;
	err = setup_rx_descbuffer(ring, desc, meta, GFP_ATOMIC);
	if (unlikely(err)) {
		b43dbg(ring->dev->wl, "DMA RX: setup_rx_descbuffer() failed\n");
		goto drop_recycle_buffer;
	}
	unmap_descbuffer(ring, dmaaddr, ring->rx_buffersize, 0);
	skb_put(skb, len + ring->frameoffset);
	skb_pull(skb, ring->frameoffset);
	b43_rx(ring->dev, skb, rxhdr);
drop:
	return;
drop_recycle_buffer:
	/* Poison and recycle the RX buffer. */
	b43_poison_rx_buffer(ring, skb);
	sync_descbuffer_for_device(ring, dmaaddr, ring->rx_buffersize);
}
"""

ground_truth_slicing = """
static void dma_rx(struct b43_dmaring *ring, int *slot)
{
    u16 len;
    len = le16_to_cpu(rxhdr->frame_len);
    if (unlikely(len > ring->rx_buffersize)) {
        s32 tmp = len;
        while (1) {
            tmp -= ring->rx_buffersize;
            if (tmp <= 0)
                break;
        }
        goto drop;
    }

    skb_put(skb, len + ring->frameoffset);
drop:
    return;
}
"""

In [17]:
# perform program slicing
focus_line = "len + ring->frameoffset > ring->rx_buffersize"
focus_var = "ring->rx_buffersize"

# 1. 获取所有和目标变量相关的代码
# 2. 获取所有和目标变量相关的代码的上下文
# 3. 获取所有和目标变量相关的代码的上下文的上下文
# 4. 获取所有和目标变量相关的代码的上下文的上下文的上下文
prompt = """
You are a program static analysis expert.

You are given a C code and a focus line of code.
Please perform program slicing on the code based on the focus line.

The output should be a list of code blocks that are related to the focus line.

Here is the C code:
{}

Here is the focus line:
{} 

Here is the focus variable:
{}

Please perform program slicing on the code based on the focus line and focus variable.
Please only output the code that is related to the focus line and focus variable.
ONLY KEEP all the code that use or change the focus variable.
REMOVE all the code that does not use or change the focus variable.

Output the result as an independent C function.
"""

output = llm_chat(prompt.format(raw_c_code, focus_line, focus_var))

# printy print the output
print(output)

```c
static void dma_rx(struct b43_dmaring *ring, int *slot)
{
	struct b43_dmadesc_generic *desc;
	struct b43_dmadesc_meta *meta;
	struct b43_rxhdr_fw4 *rxhdr;
	struct sk_buff *skb;
	u16 len;
	int err;
	dma_addr_t dmaaddr;
	desc = ring->ops->idx2desc(ring, *slot, &meta);
	skb = meta->skb;
	rxhdr = (struct b43_rxhdr_fw4 *)skb->data;
	len = le16_to_cpu(rxhdr->frame_len);
	if (unlikely(len + ring->frameoffset > ring->rx_buffersize)) {
		/* The data did not fit into one descriptor buffer
		 * and is split over multiple buffers.
		 * This should never happen, as we try to allocate buffers
		 * big enough. So simply ignore this packet.
		 */
		int cnt = 0;
		s32 tmp = len;
		while (1) {
			desc = ring->ops->idx2desc(ring, *slot, &meta);
			/* recycle the descriptor buffer. */
			b43_poison_rx_buffer(ring, meta->skb);
			sync_descbuffer_for_device(ring, meta->dmaaddr,
						   ring->rx_buffersize);
			*slot = next_slot(ring, *slot);
			cnt++;
			tmp -= ring->rx_buffersize;
			if (tmp <= 0)
		